## Introduction to Neural Networks - Fraud Detection

Nilson reports that U.S. card fraud (credit, debt, etc) was reportedly 9 billion dollars in 2016 and expected to increase to 12 billion dollars by 2020. For perspective, in 2017 both PayPal's and Mastercard's revenue was only $10.8 billion each.


**Objective:** In this session, given the credit card transactions, we will build a simple neural network (i.e., Multilayer perceptrons) for Fraud Detection using Keras.

This notebooks covers,

1. Creating a Model

2. Adding Layers

3. Activations

4. Optimizers and Loss functions

5. Evaluation

### Dataset Description

The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, the original features and more background information about the data is not provided. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Source: https://www.kaggle.com/mlg-ulb/creditcardfraud

In [1]:
!pip install tensorflow==2.0

  Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4835 sha256=ebf8454a63e8495e26c868353acaca48a13a632ad5c0d217e9bd36acfdb09ffb
  Stored in directory: C:\Users\Hp\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7546 sha256=09a7dad3a45ff1a4048c19a88052d7d73642928c242526d7cb08111618e28771
  Stored in directory: C:\Users\Hp\AppData\Local\pip\Cache\wheels\5c\2e\7e\a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built termcolor gast


In [4]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers


In [7]:
data = pd.read_csv("bank_churn.csv")

In [8]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
data = data.drop("CustomerId", axis = 1)

In [10]:
data = data.drop("RowNumber", axis = 1)

In [11]:
data = data.drop("Surname", axis = 1)

In [12]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [36]:
X_data = data.iloc[:, :-1]

In [30]:
X_data.shape

(10000, 10)

In [31]:
X_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [37]:
X_data['Gender']=np.where(X_data['Gender'].str.contains('Male'),1,0)


##cat_df_flights_specific['US_code'] = np.where(cat_df_flights_specific['carrier'].str.contains('US'), 1, 0)


In [33]:
X_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57
3,699,France,0,39,1,0.00,2,0,0,93826.63
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10


In [38]:

X_data=pd.get_dummies(X_data,columns=['Geography'],prefix=['Geography'])

#cat_df_flights_onehot = pd.get_dummies(cat_df_flights_onehot, columns=['carrier'], prefix = ['carrier'])


In [39]:
X_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1


In [40]:
y_data = data.iloc[:, -1]

In [41]:
y_data.shape

(10000,)

In [ ]:
"'In the case of this dataset The target variable is the Exited. In order to make our model optimized we have dropped the redundant data
CustomerID,RowNumber,Surname'"

In [18]:
y_data.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state = 7)

In [43]:
X_train = preprocessing.normalize(X_train)

array([[9.83888630e-01, 1.56421086e-03, 6.25684343e-02, ...,
        1.56421086e-03, 0.00000000e+00, 0.00000000e+00],
       [4.07481381e-03, 7.14879616e-06, 2.14463885e-04, ...,
        7.14879616e-06, 0.00000000e+00, 0.00000000e+00],
       [3.25738767e-03, 5.85861092e-06, 2.69496102e-04, ...,
        0.00000000e+00, 0.00000000e+00, 5.85861092e-06],
       ...,
       [4.23638988e-03, 6.19355246e-06, 2.16774336e-04, ...,
        6.19355246e-06, 0.00000000e+00, 0.00000000e+00],
       [5.37737031e-03, 7.83873224e-06, 2.66516896e-04, ...,
        0.00000000e+00, 0.00000000e+00, 7.83873224e-06],
       [4.36238487e-03, 6.01708258e-06, 1.56444147e-04, ...,
        6.01708258e-06, 0.00000000e+00, 0.00000000e+00]])

In [47]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 12)
(2000, 12)
(8000,)
(2000,)


### Creating a model

Keras model object can be created with Sequential class

At the outset, the model is empty per se. It is completed by adding additional layers and compilation


In [55]:
model = Sequential()


### Adding layers [layers and activations]

Keras layers can be added to the model

Adding layers are like stacking lego blocks one by one

It should be noted that as this is a classification problem, sigmoid layer (softmax for multi-class problems) should be added


In [56]:
model.add(Dense(64, input_shape = (12,), activation = 'relu'))
model.add(Dense(32, activation = 'tanh'))
model.add(Dense(1, activation = 'sigmoid'))

### Model compile [optimizers and loss functions]

Keras model should be "compiled" prior to training

Types of loss (function) and optimizer should be designated


In [57]:
sgd = optimizers.Adam(lr = 0.001)s

In [58]:
model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics=['accuracy'])

### Summary of the model

In [59]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


### Training [Forward pass and Backpropagation]

Training the model

In [81]:
model.fit(X_train, y_train.values, batch_size = 50, epochs = 20, verbose = 1)

Train on 8000 samples
Epoch 1/20
8000/8000 [==============================] - 0s 57us/sample - loss: 0.4984 - accuracy: 0.7968
Epoch 2/20
8000/8000 [==============================] - 0s 28us/sample - loss: 0.4984 - accuracy: 0.7968
Epoch 3/20
8000/8000 [==============================] - 0s 27us/sample - loss: 0.4989 - accuracy: 0.7968
Epoch 4/20
8000/8000 [==============================] - 0s 27us/sample - loss: 0.4985 - accuracy: 0.7968
Epoch 5/20
8000/8000 [==============================] - 0s 26us/sample - loss: 0.4986 - accuracy: 0.7968
Epoch 6/20
8000/8000 [==============================] - 0s 26us/sample - loss: 0.4983 - accuracy: 0.7968
Epoch 7/20
8000/8000 [==============================] - 0s 26us/sample - loss: 0.4985 - accuracy: 0.7968
Epoch 8/20
8000/8000 [==============================] - 0s 31us/sample - loss: 0.4986 - accuracy: 0.7968
Epoch 9/20
8000/8000 [==============================] - 0s 30us/sample - loss: 0.4983 - accuracy: 0.7968
Epoch 10/20
8000/8000 [==========

### Evaluation
Keras model can be evaluated with evaluate() function

Evaluation results are contained in a list



In [82]:
X_test = preprocessing.normalize(X_test)

In [83]:
results = model.evaluate(X_test, y_test.values)


2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [84]:
print(model.metrics_names)
print(results)    

['loss', 'accuracy']
[0.4980484414100647, 0.7945]


In [85]:
result=model.predict(X_test)


In [96]:
X_predict=[]
for val in result:        
      if(val <0.5):
        print("Not Churn")
      else:
        print("Churn")


Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn


Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn
Not Churn


[]

### Confusion Matrix

In [73]:
Y_pred_cls = model.predict_classes(X_test, batch_size=200, verbose=0)
print('Accuracy Model1 (Dropout): '+ str(model.evaluate(X_test,y_test.values)[1]))
print('Recall_score: ' + str(recall_score(y_test.values,Y_pred_cls)))
print('Precision_score: ' + str(precision_score(y_test.values, Y_pred_cls)))
print('F-score: ' + str(f1_score(y_test.values,Y_pred_cls)))
confusion_matrix(y_test.values, Y_pred_cls)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Accuracy Model1 (Dropout): 0.7945
Recall_score: 0.0
Precision_score: 0.0
F-score: 0.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


array([[1589,    0],
       [ 411,    0]], dtype=int64)

#### Feel free to experiment with the model and get to better evaluation metric scores. 
Happy Learning!